In [3]:
import numpy as np
import pandas as pd
import feather

from google.cloud import bigquery                                                                                
from google.oauth2 import service_account                                                                        
import pandas as pd                                                                                              
import os                                                                                                                                                                                                                           
# User's credentials and project_id                                                                              
project_id = 'circle-ds-pipelines'                                                                                                                                                                                                  
# Construct a BigQuery client object.                                                                            
client = bigquery.Client(project=project_id)    


/Users/gordon.liao/anaconda3/lib/python3.11/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [ ]:
sql="""SELECT TIMESTAMP_TRUNC(timestamp,DAY) as date,SUM(price * volume_24h) / SUM(volume_24h) as price,
array_agg(volume_24h order by timestamp desc limit 1)[OFFSET(0)] as volume, 
array_agg(market_cap order by timestamp desc limit 1)[OFFSET(0)] as marketcap                                                                             
FROM circle-ds-pipelines.prices.cmc_stablecoins where token='USDC' group by TIMESTAMP_TRUNC(timestamp,DAY);"""

# sql= """SELECT  * from circle-ds-pipelines.prices.cmc_stablecoins where token='USDC'; """
query_job = client.query(sql)                                                                                                                                                                          
results = query_job.result()                                                                                     
stb = results.to_dataframe()
stb=stb.set_index('date').sort_index()
stb['volume'] = pd.to_numeric(stb['volume'])
stb['marketcap'] = pd.to_numeric(stb['marketcap'])
stb['price'] = pd.to_numeric(stb['price'])
stb.index = stb.index.tz_localize(None)

stb.to_feather('~/Documents/Data/cmc_usdc_marketcap.feather')

In [55]:
import pandas as pd                                                                                             
import pandas_datareader.data as web                                          
from datetime import datetime                                                   
                                                                                                                   
bitcoin_ticker = 'CBBTCUSD' 
start_date = '2014-12-01' # FRED's Bitcoin data starts from 
end_date = datetime.now().strftime('%Y-%m-%d')  
                                                                                                                   
                             
bitcoin_df = web.DataReader(bitcoin_ticker, 'fred', start_date, end_date)                                       
bitcoin_df.to_feather('~/Documents/Data/btc_price_fred.feather')


/Users/gordon.liao/anaconda3/lib/python3.11/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


NameError: name 'pd' is not defined

In [4]:
sql="""SELECT TIMESTAMP_TRUNC(timestamp,DAY) as date,SUM(price * volume_24h) / SUM(volume_24h) as price,
array_agg(volume_24h order by timestamp desc limit 1)[OFFSET(0)] as volume, 
array_agg(market_cap order by timestamp desc limit 1)[OFFSET(0)] as marketcap                                                                             
FROM circle-ds-pipelines.prices.cmc_stablecoins where token='USDT' group by TIMESTAMP_TRUNC(timestamp,DAY);"""

# sql= """SELECT  * from circle-ds-pipelines.prices.cmc_stablecoins where token='USDT'; """
query_job = client.query(sql)                                                                                                                                                                          
results = query_job.result()                                                                                     
stb = results.to_dataframe()
stb=stb.set_index('date').sort_index()
stb['volume'] = pd.to_numeric(stb['volume'])
stb['marketcap'] = pd.to_numeric(stb['marketcap'])
stb['price'] = pd.to_numeric(stb['price'])
stb.index = stb.index.tz_localize(None)

stb.to_feather('~/Documents/Data/cmc_usdt_marketcap.feather')

/Users/gordon.liao/anaconda3/lib/python3.11/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [4]:


sql=""" SELECT
  ds,
  token,
  SUM(transactions_28_day) AS transactions_28_day,
  SUM(gross_transfers_28_day) AS gross_transfers_28_day,
  SUM(net_transfers_28_day) AS net_transfers_28_day
  FROM
  `circle-ds-pipelines.multichain.dim_address_stablecoins`
WHERE
  token IN ('USDC', 'USDT')
  AND ds >= '2021-01-01'
GROUP BY
  ds, token """
  
query_job = client.query(sql)                                                                                                                                                                          
results = query_job.result()                                                                                     
df_stableactivities = results.to_dataframe()
df_stableactivities['date']=pd.to_datetime(df_stableactivities['ds'])
df_stableactivities = df_stableactivities.drop(columns=['ds'])
df_stableactivities=df_stableactivities.set_index('date').sort_index()

df_stableactivities.to_feather('~/Documents/Data/stableactivities_monthly.feather')

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))